In [ ]:
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor, Compose, Normalize
from torch.utils.data import DataLoader

device = 'cuda'

transform = Compose([ToTensor(), Normalize((0.1307,), (0.3081,))])

datasets = {
    'train': MNIST(root='data', train=True, download=True, transform=transform),
    'test': MNIST(root='data', train=False, download=True, transform=transform)
}

loaders = {
    'train': DataLoader(dataset=datasets['train'],batch_size=64,shuffle=True, pin_memory=True, pin_memory_device=device, num_workers=2),
    'test': DataLoader(dataset=datasets['test'],batch_size=64,shuffle=False, pin_memory=True, pin_memory_device=device, num_workers=2)
}

In [ ]:
from torch import Tensor
from torch.nn import Module
from torch.nn import Flatten,  Sequential
from torch.nn import Linear, ReLU, Dropout
from model.dropconnect import DropConnectLinear, DropConnectBatchAverage

class Perceptron(Module):
    def __init__(self, input_features: int, hidden_dimension: int, output_features: int, p: float):
        super().__init__()
        self.flatten = Flatten()
        self.layers = Sequential(
            Linear(input_features, hidden_dimension),
            ReLU(),
            Dropout(p),
            Linear(hidden_dimension, output_features),
        )

    def forward(self, input: Tensor) -> Tensor:
        input = self.flatten(input)
        return self.layers(input)
            

class DropConnectPerceptron(Module):
    def __init__(self, input_features: int, hidden_dimension: int, output_features: int, p: float):
        super().__init__()
        self.flatten = Flatten()
        self.layers = Sequential(
            DropConnectLinear(input_features, hidden_dimension, p=p),
            ReLU(),
            DropConnectBatchAverage(),
            DropConnectLinear(hidden_dimension, output_features, p=p),
            ReLU(),
            DropConnectBatchAverage()
        )

    def forward(self, input: Tensor) -> Tensor:
        input = self.flatten(input)
        return self.layers(input)

In [ ]:
import torch
from torch.optim import SGD, Adam
from torch.nn import CrossEntropyLoss
from utils import train, test, Metrics

In [ ]:
from typing import Dict
from typing import Optional
from matplotlib.pyplot import figure, show
from matplotlib.axes import Axes

def plot(metrics: Dict[str, Metrics], metric: str, ax: Optional[Axes] = None) -> Axes:
    if ax is None:
        plot = figure()
        ax = plot.add_subplot()

    for key, value in metrics.items():
        ax.plot(value.history[metric], label=key)
    ax.legend()
    ax.set_title(metric)
    ax.set_xlabel('epoch')
    ax.set_ylabel(metric)
    return ax

In [ ]:
model = DropConnectPerceptron(28*28, 256, 10, 0.2).to(device)
criterion = CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)

torch.set_float32_matmul_precision('high')
torch.compile(model)

metrics = {
    'train': Metrics(),
    'test': Metrics()
}

for epoch in range(1, 30):
    train(model, criterion, optimizer, loaders['train'], metrics['train'], device)
    test(model, criterion, loaders['test'], metrics['test'], device)


plot(metrics, 'loss')
show()

plot(metrics, 'accuracy')
show()

for metric in metrics.values():
    metric.reset()

In [ ]:
model = Perceptron(28*28, 256, 10, 0.2).to(device)
criterion = CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)

torch.set_float32_matmul_precision('high')
torch.compile(model)

metrics = {
    'train': Metrics(),
    'test': Metrics()
}

for epoch in range(1, 30):
    train(model, criterion, optimizer, loaders['train'], metrics['train'], device)
    test(model, criterion, loaders['test'], metrics['test'], device)


plot(metrics, 'loss')
show()

plot(metrics, 'accuracy')
show()

for metric in metrics.values():
    metric.reset()

In [ ]:
model = DropConnectPerceptron(28*28, 256, 10, 0.5).to(device)
criterion = CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)

torch.set_float32_matmul_precision('high')
torch.compile(model)

metrics = {
    'train': Metrics(),
    'test': Metrics()
}

for epoch in range(1, 30):
    train(model, criterion, optimizer, loaders['train'], metrics['train'], device)
    test(model, criterion, loaders['test'], metrics['test'], device)


plot(metrics, 'loss')
show()

plot(metrics, 'accuracy')
show()

for metric in metrics.values():
    metric.reset()

In [ ]:
model = Perceptron(28*28, 256, 10, 0.5).to(device)
criterion = CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)

torch.set_float32_matmul_precision('high')
torch.compile(model)

metrics = {
    'train': Metrics(),
    'test': Metrics()
}

for epoch in range(1, 30):
    train(model, criterion, optimizer, loaders['train'], metrics['train'], device)
    test(model, criterion, loaders['test'], metrics['test'], device)


plot(metrics, 'loss')
show()

plot(metrics, 'accuracy')
show()

for metric in metrics.values():
    metric.reset()

In [ ]:
model = DropConnectPerceptron(28*28, 512, 10, 0.2).to(device)
criterion = CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)

torch.set_float32_matmul_precision('high')
torch.compile(model)

metrics = {
    'train': Metrics(),
    'test': Metrics()
}

for epoch in range(1, 30):
    train(model, criterion, optimizer, loaders['train'], metrics['train'], device)
    test(model, criterion, loaders['test'], metrics['test'], device)


plot(metrics, 'loss')
show()

plot(metrics, 'accuracy')
show()

for metric in metrics.values():
    metric.reset()

In [ ]:
model = Perceptron(28*28, 512, 10, 0.2).to(device)
criterion = CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)

torch.set_float32_matmul_precision('high')
torch.compile(model)

metrics = {
    'train': Metrics(),
    'test': Metrics()
}

for epoch in range(1, 30):
    train(model, criterion, optimizer, loaders['train'], metrics['train'], device)
    test(model, criterion, loaders['test'], metrics['test'], device)


plot(metrics, 'loss')
show()

plot(metrics, 'accuracy')
show()

for metric in metrics.values():
    metric.reset()

In [ ]:
model = DropConnectPerceptron(28*28, 512, 10, 0.5).to(device)
criterion = CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)

torch.set_float32_matmul_precision('high')
torch.compile(model)

metrics = {
    'train': Metrics(),
    'test': Metrics()
}

for epoch in range(1, 30):
    train(model, criterion, optimizer, loaders['train'], metrics['train'], device)
    test(model, criterion, loaders['test'], metrics['test'], device)


plot(metrics, 'loss')
show()

plot(metrics, 'accuracy')
show()

for metric in metrics.values():
    metric.reset()

In [ ]:
model = Perceptron(28*28, 512, 10, 0.5).to(device)
criterion = CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)

torch.set_float32_matmul_precision('high')
torch.compile(model)

metrics = {
    'train': Metrics(),
    'test': Metrics()
}

for epoch in range(1, 30):
    train(model, criterion, optimizer, loaders['train'], metrics['train'], device)
    test(model, criterion, loaders['test'], metrics['test'], device)


plot(metrics, 'loss')
show()

plot(metrics, 'accuracy')
show()

for metric in metrics.values():
    metric.reset()

In [ ]:
model = DropConnectPerceptron(28*28, 256, 10, 0.2).to(device)
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.01)

torch.set_float32_matmul_precision('high')
torch.compile(model)

metrics = {
    'train': Metrics(),
    'test': Metrics()
}

for epoch in range(1, 30):
    train(model, criterion, optimizer, loaders['train'], metrics['train'], device)
    test(model, criterion, loaders['test'], metrics['test'], device)


plot(metrics, 'loss')
show()

plot(metrics, 'accuracy')
show()

for metric in metrics.values():
    metric.reset()

In [ ]:
model = Perceptron(28*28, 256, 10, 0.2).to(device)
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.01)

torch.set_float32_matmul_precision('high')
torch.compile(model)

metrics = {
    'train': Metrics(),
    'test': Metrics()
}

for epoch in range(1, 30):
    train(model, criterion, optimizer, loaders['train'], metrics['train'], device)
    test(model, criterion, loaders['test'], metrics['test'], device)


plot(metrics, 'loss')
show()

plot(metrics, 'accuracy')
show()

for metric in metrics.values():
    metric.reset()

In [ ]:
model = DropConnectPerceptron(28*28, 256, 10, 0.5).to(device)
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.01)

torch.set_float32_matmul_precision('high')
torch.compile(model)

metrics = {
    'train': Metrics(),
    'test': Metrics()
}

for epoch in range(1, 30):
    train(model, criterion, optimizer, loaders['train'], metrics['train'], device)
    test(model, criterion, loaders['test'], metrics['test'], device)


plot(metrics, 'loss')
show()

plot(metrics, 'accuracy')
show()

for metric in metrics.values():
    metric.reset()

In [ ]:
model = Perceptron(28*28, 256, 10, 0.5).to(device)
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.01)

torch.set_float32_matmul_precision('high')
torch.compile(model)

metrics = {
    'train': Metrics(),
    'test': Metrics()
}

for epoch in range(1, 30):
    train(model, criterion, optimizer, loaders['train'], metrics['train'], device)
    test(model, criterion, loaders['test'], metrics['test'], device)


plot(metrics, 'loss')
show()

plot(metrics, 'accuracy')
show()

for metric in metrics.values():
    metric.reset()

In [ ]:
model = DropConnectPerceptron(28*28, 512, 10, 0.2).to(device)
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.01)

torch.set_float32_matmul_precision('high')
torch.compile(model)

metrics = {
    'train': Metrics(),
    'test': Metrics()
}

for epoch in range(1, 30):
    train(model, criterion, optimizer, loaders['train'], metrics['train'], device)
    test(model, criterion, loaders['test'], metrics['test'], device)


plot(metrics, 'loss')
show()

plot(metrics, 'accuracy')
show()

for metric in metrics.values():
    metric.reset()

In [ ]:
model = Perceptron(28*28, 512, 10, 0.2).to(device)
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.01)

torch.set_float32_matmul_precision('high')
torch.compile(model)

metrics = {
    'train': Metrics(),
    'test': Metrics()
}

for epoch in range(1, 30):
    train(model, criterion, optimizer, loaders['train'], metrics['train'], device)
    test(model, criterion, loaders['test'], metrics['test'], device)


plot(metrics, 'loss')
show()

plot(metrics, 'accuracy')
show()

for metric in metrics.values():
    metric.reset()

In [ ]:
model = DropConnectPerceptron(28*28, 512, 10, 0.5).to(device)
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.01)

torch.set_float32_matmul_precision('high')
torch.compile(model)

metrics = {
    'train': Metrics(),
    'test': Metrics()
}

for epoch in range(1, 30):
    train(model, criterion, optimizer, loaders['train'], metrics['train'], device)
    test(model, criterion, loaders['test'], metrics['test'], device)


plot(metrics, 'loss')
show()

plot(metrics, 'accuracy')
show()

for metric in metrics.values():
    metric.reset()

In [ ]:
model = Perceptron(28*28, 512, 10, 0.5).to(device)
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.01)

torch.set_float32_matmul_precision('high')
torch.compile(model)

metrics = {
    'train': Metrics(),
    'test': Metrics()
}

for epoch in range(1, 30):
    train(model, criterion, optimizer, loaders['train'], metrics['train'], device)
    test(model, criterion, loaders['test'], metrics['test'], device)


plot(metrics, 'loss')
show()

plot(metrics, 'accuracy')
show()

for metric in metrics.values():
    metric.reset()